In [43]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: haiyi1
"""

import torch
import numpy as np
import os
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models, transforms
from torch.optim.lr_scheduler import StepLR
from google.colab import drive
drive.mount('/content/drive')

class cnn3D(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, h_dim3, dim_1d):
        super(cnn3D, self).__init__()
        self.conv1 = self.cnn_layer(x_dim, h_dim1)
        self.cnn_bn1 = nn.BatchNorm3d(h_dim1)
        self.conv2 = self.cnn_layer(h_dim1, h_dim2)
        self.cnn_bn2 = nn.BatchNorm3d(h_dim2)
        # self.conv3 = self.cnn_layer(h_dim2, h_dim3)
        # self.cnn_bn3 = nn.BatchNorm3d(h_dim3)

        self.fc1 = nn.Linear(3*6*6*h_dim2 + dim_1d, 8*dim_1d)
        self.Lrelu = nn.LeakyReLU()
        self.fc_bn = nn.BatchNorm1d(8*dim_1d)
        self.fc2 = nn.Linear(8*dim_1d, 4*dim_1d)
        self.fc_bn2 = nn.BatchNorm1d(4*dim_1d)
        self.fc3 = nn.Linear(4*dim_1d, dim_1d)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.5)



    def cnn_layer(self, channel_in, channel_out):
        cnn3D_layer = nn.Sequential(nn.Conv3d(channel_in, channel_out, kernel_size=(3, 3, 3), stride=1, padding='same'),
                             nn.LeakyReLU(),
                             nn.MaxPool3d(kernel_size=(2, 2, 2), stride = 2),
                             )
        return cnn3D_layer

    def forward(self, x, x_1d):
        #print('input shape', x.shape)
        x = self.conv1(x)
        x = self.cnn_bn1(x)
        x = self.conv2(x)
        x = self.cnn_bn2(x)
        # x = self.conv3(x)
        # x = self.cnn_bn3(x)
        x = x.view(x.size(0), -1)
        x_concat = torch.cat([x, x_1d], 1)
        x = self.fc1(x_concat)
        x = self.Lrelu(x)
        x = self.fc_bn(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.fc_bn2(x)
        x = self.drop(x)
        x = self.fc3(x)
        x = self.relu(x)

        return x

# self-define dataset
class customData():
    def __init__(self, files, transform = None):
        data = np.load(str(files))
        depth = data["Depth"]
        permx = data["Permx"]
        permy = data["Permy"]
        permz = data["Permz"]
        poro = data["Poro"]
        porv = data["Porv"]
        satnum = data["Satnum"]
        tranx = data["Tranx"]
        trany = data["Trany"]
        tranz = data["Tranz"]

        inp_3d1 = np.stack((permx, permy), axis = 1)
        inp_3d2 = np.stack((poro, porv), axis = 1)
        inp_3d3 = np.stack((tranx, trany), axis = 1)
        inp_3d4 = np.stack((depth, satnum), axis = 1)
        inp_3d5 = np.stack((permz, tranz), axis = 1)

        inp_3d = np.concatenate((inp_3d1, inp_3d2), axis = 1)
        inp_3d = np.concatenate((inp_3d, inp_3d3), axis = 1)
        inp_3d = np.concatenate((inp_3d, inp_3d4), axis = 1)
        inp_3d = np.concatenate((inp_3d, inp_3d5), axis = 1)

        inp_1d = data["inp1D"]
        out_1d = data["out1D"]

        self.inp_3d = torch.as_tensor(inp_3d).to(torch.float32)
        self.inp_1d = torch.as_tensor(inp_1d).to(torch.float32)
        self.out_1d = torch.as_tensor(out_1d).to(torch.float32)
        #print(inp_3d.shape, inp_1d.shape, out_1d.shape)

    def __getitem__(self, idx):
        inp1 = self.inp_3d[idx]
        inp2 = self.inp_1d[idx]
        label = self.out_1d[idx]

        return inp1, inp2, label

    def __len__(self):
        return len(self.out_1d)

def train(model, device, train_loader, optimizer, epoch, train_size):
    model.train()
    train_loss = 0
    for batch_idx, (inp3d, inp1d, true_y) in enumerate(train_loader):
        inp3d, inp1d, true_y = inp3d.to(device), inp1d.to(device), true_y.to(device)
        optimizer.zero_grad()
        pred_y = model(inp3d, inp1d)
        mse_loss = torch.nn.MSELoss()
        loss = mse_loss(true_y, pred_y)
        train_loss += loss.item()
        #print(loss.item())
        loss.backward()
        optimizer.step()

    print('==> Epoch: {} Ave Train loss: {:.6f} Train loss: {:.6f}'.format(epoch, train_loss/batch_idx, loss.item()))
    return train_loss/batch_idx

def valid(model, device, valid_loader, epoch):
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for inp3d, inp1d, true_y in valid_loader:
            inp3d, inp1d, true_y = inp3d.to(device), inp1d.to(device), true_y.to(device)
            pred_y = model(inp3d, inp1d)
            mse_loss = torch.nn.MSELoss()
            valid_loss += mse_loss(pred_y, true_y)

    #valid_loss /= len(valid_loader.dataset)
    print('==> Epoch: {} Ave Validation loss: {:.6f}'.format(epoch, valid_loss))


def test(model, device, test_loader, epoch, test_size):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inp3d, inp1d, true_y in test_loader:
            inp3d, inp1d, true_y = inp3d.to(device), inp1d.to(device), true_y.to(device)
            pred_y = model(inp3d, inp1d)
            mse_loss = torch.nn.MSELoss()
            #print(mse_loss(pred_y, true_y))
            test_loss += mse_loss(pred_y, true_y)

    #test_loss /= len(test_loader.dataset)
    print('==> Epoch: {} Ave Test loss: {:.6f}'.format(epoch, test_loss))
    return test_loss, pred_y, true_y

def main():
    # Training settings
    batch_size = 50
    n_epoch = 1000
    device = "cuda:0"
    #device = "cpu"
    lr = 1e-4
    h_dim1 = 16
    h_dim2 = 32
    h_dim3 = 64
    dim_1d = 300
    x_dim = 10
    train_size = 450
    test_size = 25
    max_WOPR = 31796.140625
    min_WOPR = 0.8107147216796875
    #svae_dir = ''
    train_path = "/content/drive/MyDrive/well1/train/"
    valid_path = "/content/drive/MyDrive/well1/valid/"
    test_path = "/content/drive/MyDrive/well1/test/"

    train_name = 'train_well7.npz'
    valid_name = 'valid_well7.npz'
    test_name = 'test_well7.npz'
    train_files = train_path + train_name
    valid_files = valid_path + valid_name
    test_files = test_path + test_name

    train_dataset = customData(train_files, transform = None)
    test_dataset = customData(test_files, transform = None)
    valid_dataset = customData(valid_files, transform = None)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

    cnn_3d = cnn3D(x_dim, h_dim1, h_dim2, h_dim3, dim_1d)
    model = cnn_3d.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    scheduler = StepLR(optimizer, step_size=200, gamma=0.8)
    loss_train = []
    loss_test = []
    for epoch in range(1, n_epoch):
        train_loss = train(model, device, train_loader, optimizer, epoch, train_size)
        valid(model, device, valid_loader, epoch)
        test_loss, pred_y, true_y = test(model, device, test_loader, epoch, test_size)
        scheduler.step()
        loss_train.append((epoch, train_loss))
        test_loss = test_loss.cpu()
        loss_test.append((epoch, test_loss))

    # unnormalized

    loss_train = np.array(loss_train)
    loss_test = np.array(loss_test)

    pred_y = pred_y.to('cpu').numpy() * (max_WOPR - min_WOPR) + min_WOPR
    true_y = true_y.to('cpu').numpy() * (max_WOPR - min_WOPR) + min_WOPR

    np.save('prediction7', pred_y)
    np.save('true_y7', true_y)
    np.save('loss_train7', loss_train)
    np.save('loss_test7', loss_test)

    torch.save(model.state_dict(), "3DCNN7.pt")

if __name__ == '__main__':
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
==> Epoch: 1 Ave Train loss: 0.316894 Train loss: 0.265571
==> Epoch: 1 Ave Validation loss: 0.017182
==> Epoch: 1 Ave Test loss: 0.019690
==> Epoch: 2 Ave Train loss: 0.290720 Train loss: 0.253330
==> Epoch: 2 Ave Validation loss: 0.036751
==> Epoch: 2 Ave Test loss: 0.040445
==> Epoch: 3 Ave Train loss: 0.269329 Train loss: 0.228618
==> Epoch: 3 Ave Validation loss: 0.101233
==> Epoch: 3 Ave Test loss: 0.105567
==> Epoch: 4 Ave Train loss: 0.246779 Train loss: 0.215693
==> Epoch: 4 Ave Validation loss: 0.152883
==> Epoch: 4 Ave Test loss: 0.156627
==> Epoch: 5 Ave Train loss: 0.230648 Train loss: 0.204725
==> Epoch: 5 Ave Validation loss: 0.138956
==> Epoch: 5 Ave Test loss: 0.142169
==> Epoch: 6 Ave Train loss: 0.212169 Train loss: 0.189652
==> Epoch: 6 Ave Validation loss: 0.095701
==> Epoch: 6 Ave Test loss: 0.098761
==> Epoch: 7 Ave Train loss: 0.199489